In [6]:
from langchain.document_loaders import BSHTMLLoader
from bs4 import BeautifulSoup
import os
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders.csv_loader import CSVLoader
## reference
##https://colab.research.google.com/drive/1gyGZn_LZNrYXYXa-pltFExbptIe7DAPe?usp=sharing#scrollTo=wKfX4vX-5RFT

In [ ]:
# read in files

folder_path = '/Users/linqianpeng/Documents/immigration/data/canadian_government/Family sponsorship'

## List all files in the specified directory
all_files = os.listdir(folder_path)

## Filter out only files that have .html extension
html_files = [file for file in all_files if file.endswith('.html')]

In [82]:
## loading data from canada government html files folder

loader = DirectoryLoader(folder_path, glob="**/*.html")
docs = loader.load()

In [83]:
print(len(docs),len(html_files))

11 11


In [85]:
openai_api_key = os.environ.get('OPENAI_API_KEY')
print(len(openai_api_key))

51


In [115]:
## load data from reddit forum in csv file 

loader = CSVLoader(file_path="reddit_results.csv")

data = loader.load()

In [86]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(docs)

In [95]:
len(texts)

405

In [116]:
#splitting the reddit forum texts
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(data)
print(len(texts))

1119


In [69]:
## vectoriza

In [7]:
persist_directory = 'docs/chroma/canadiangovernment/'

In [118]:

embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(texts, embeddings,persist_directory=persist_directory)

In [119]:
## here we are using OpenAI embeddings but in future we will swap out to local embeddings
embedding = OpenAIEmbeddings()

vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [120]:
print(vectordb._collection.count())

2813


In [94]:
##html file
print(vectordb._collection.count())

575


In [121]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [8]:
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding)

In [9]:
print(vectordb._collection.count())

2813


In [124]:
retriever = vectordb.as_retriever()
docs = retriever.get_relevant_documents("what are the requirement to apply for common law?")

In [12]:
from langchain import HuggingFaceHub, PromptTemplate, LLMChain
huggingfacehub_api_token = os.environ['HUGGINGFACEHUB_API_TOKEN']
repo_id = "tiiuae/falcon-7b-instruct"

repo_id = "meta-llama/Llama-2-7b-chat-hf"
llm = HuggingFaceHub(huggingfacehub_api_token=huggingfacehub_api_token, 
                     repo_id=repo_id, 
                     model_kwargs={"temperature":0.6, "max_new_tokens":500})

/Users/linqianpeng/Documents/IsThisNormal/isthisnormal/lib/python3.9/site-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [14]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
retriever = vectordb.as_retriever()
qa_chain = RetrievalQA.from_chain_type(llm=llm,#OpenAI(), 
                                  chain_type="stuff", 
                                  retriever=retriever, 
                                  return_source_documents=True)

In [15]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [16]:
# full example
query = "what can i use as a letter of employment if I currently do not have a job"
llm_response = qa_chain(query)
process_llm_response(llm_response)



It is important to note that you should not use a letter of employment from your current employer as your main reference. Instead, you should use a letter of employment from a previous employer. However, if you do not have a previous employer, you can use a letter of recommendation from a professional or a friend. 

Key points to keep in mind:

1. The letter should be on your company's letterhead or from a reputable organization.
2. The letter should address the hiring manager or HR representative.
3. The letter should be concise and to the point.
4. The letter should provide information about your previous job duties and your qualifications.
5. The letter should be from someone other than a family member.
6. The letter should be from someone who knows you well and can provide a positive reference.


Sources:
reddit_results.csv
reddit_results.csv
reddit_results.csv
reddit_results.csv


In [19]:
repo_id = "TheBloke/Wizard-Vicuna-30B-Superhot-8K-fp16"
llm = HuggingFaceHub(huggingfacehub_api_token=huggingfacehub_api_token, 
                     repo_id=repo_id, 
                     model_kwargs={"temperature":0.6, "max_new_tokens":500})

In [20]:
# full example
query = "what are the requirement to apply for common law?"
llm_response = qa_chain(query)
process_llm_response(llm_response)



The requirements to apply for common law vary depending on the country you are currently in. You should check with the authorities in that country to ensure you meet their specific requirements before applying.

For Canada, the following are the requirements:

1. You must have a valid temporary resident status.

2. You must be cohabiting with your spouse or common-law partner.

3. You must have a joint bank account, trust, or credit card.

4. You must have a valid passport or other document that proves your identity.

5. You must have submitted an application for permanent residency.

6. You must have received a letter of sponsorship from the government.

7. You must be in a genuine conjugal relationship that has been in place for at least one year.

8. You must have a valid travel document or passport.

9. You must have a letter of co-signer who is willing to take responsibility for the sponsorship.

10. You must have sufficient funds to support yourself and your family.

11. You mu